In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [ ]:
%run ./plot_end_events_for_outages_2b2_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
#TODO Maybe make small class/container to hold dataset information?
datasets = ['outg_full', 'outg_drct', 'no_outg', 'no_outg_prstn']
dataset_subdirs = {
    'outg_full':     r'Outgs_Full', 
    'outg_drct':     r'Outgs_Drct', 
    'no_outg':       r'NoOutgs', 
    'no_outg_prstn': r'NoOutgs_Pristine'
}
dataset_naming_tags = {
    'outg_full':     None, 
    'outg_drct':     '_prim_strict', 
    'no_outg':       '_no_outg', 
    'no_outg_prstn': '_no_outg_prstn'
}
dataset_is_no_outage = {
    'outg_full':     False, 
    'outg_drct':     False, 
    'no_outg':       True, 
    'no_outg_prstn': True
}
assert(len(set(datasets).symmetric_difference(set(dataset_subdirs.keys())))==0)
assert(len(set(datasets).symmetric_difference(set(dataset_naming_tags.keys())))==0)
assert(len(set(datasets).symmetric_difference(set(dataset_is_no_outage.keys())))==0)

In [ ]:
run_testing_data=False
#-------------------------
# run_date = '20221014' # Date of data acquisition
# run_date = '20221216' # Date of data acquisition
# run_date = '20230301' # Date of data acquisition
run_date = '20231221' # Date of data acquisition
#-------------------------
date_0 = '2023-04-01' # Lower limit for end events
date_1 = '2023-11-30' # Upper limit for end events
#-------------------------
# dataset = 'outg_full'
# dataset = 'outg_drct'
dataset = 'no_outg_prstn'
# dataset = 'no_outg'
#-------------------------

In [ ]:
dataset_subdirs[dataset]

In [ ]:
assert(dataset in datasets)
#-------------------------
local_or_cloud_data_dir = Utilities.get_local_data_dir()
# local_or_cloud_data_dir = r'U:\CloudData'
#-------------------------
date_pd_subdir = f"{date_0.replace('-','')}_{date_1.replace('-','')}"
data_base_dir = os.path.join(
    local_or_cloud_data_dir, 
    r'dovs_and_end_events_data', 
    run_date, 
    date_pd_subdir, 
    dataset_subdirs[dataset]
)
#-------------------------
assert(os.path.isdir(data_base_dir))
#-------------------------
files_dir = os.path.join(data_base_dir, 'EndEvents')
naming_tag = dataset_naming_tags[dataset]
is_no_outage = dataset_is_no_outage[dataset]
#-------------------------
assert(os.path.isdir(files_dir))
#-------------------------
print(f'data_base_dir = {data_base_dir}')
print(f'files_dir     = {files_dir}')
print(f'naming_tag    = {naming_tag}')
print(f'is_no_outage  = {is_no_outage}')

In [ ]:
data_base_dir

In [ ]:
# Currently, expecting grp_by_col to be 'outg_rec_nb', 'trsf_pole_nb', or ['outg_rec_nb', 'trsf_pole_nb']
#   Actually, 'outg_rec_nb' will probably not be run again, instead will likely always be ['outg_rec_nb', 'trsf_pole_nb']

In [ ]:
# run_testing_data=False
# fig_num=0

# data_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
# if run_testing_data:
#     data_base_dir = os.path.join(data_base_dir, r'TESTING_DATASETS')

#-------------------------
# files_dir = os.path.join(data_base_dir, r'EndEvents')
# naming_tag=None
# batch_size=1000
# is_no_outage = False
# grp_by_col = 'outg_rec_nb'
# grp_by_col = 'trsf_pole_nb'
# grp_by_col = ['outg_rec_nb', 'trsf_pole_nb']
#-------------------------
#-------------------------
# files_dir = os.path.join(data_base_dir, r'EndEvents_prim_strict')
# naming_tag='_prim_strict'
# # batch_size=50
# batch_size=1000
# is_no_outage = False
# # grp_by_col = 'outg_rec_nb'
# # grp_by_col = 'trsf_pole_nb'
# grp_by_col = ['outg_rec_nb', 'trsf_pole_nb']
#-------------------------
#-------------------------
# files_dir = os.path.join(data_base_dir, r'EndEvents_NoOutg')
# naming_tag='_no_outg'
batch_size=1000
# is_no_outage = True
# grp_by_col = 'trsf_pole_nb'
grp_by_col = ['trsf_pole_nb', 'no_outg_rec_nb']
#------------------------- 
#-------------------------

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None

assert_all_cols_equal=False #Seems new EndEvents have curr_acct_cls_cd as well...
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False

# 1-6, 6-11, 11-16, 16-21, 21-26, 26-31
days_min_outg_td_window=1
days_max_outg_td_window=6

if include_normalize_by_nSNs and not return_normalized_separately:
    normalize_by_nSNs_included=True
else:
    normalize_by_nSNs_included=False
    
xfmr_equip_typ_nms_of_interest = ['TRANSFORMER, OH', 'TRANSFORMER, UG']

#--------------------------------------------------
if naming_tag is None:
    naming_tag=''
    
# only set up for outg_rec_nb or trsf_pole_nb currently
assert(grp_by_col=='outg_rec_nb' or 
       grp_by_col==['outg_rec_nb', 'trsf_pole_nb'] or 
       grp_by_col=='trsf_pole_nb' or 
       grp_by_col==['trsf_pole_nb', 'no_outg_rec_nb'])

# Not possible for have outg_rec_nb for no outage case!
if is_no_outage:
    assert(grp_by_col!='outg_rec_nb' and grp_by_col!=('outg_rec_nb', 'trsf_pole_nb'))

In [ ]:
save_dfs_to_pkl = True
read_dfs_from_pkl = False
assert(save_dfs_to_pkl+read_dfs_from_pkl <=1) # Should never both read and write!

save_subdir_pkls = 'rcpo_dfs'
if grp_by_col==['outg_rec_nb', 'trsf_pole_nb']:
    save_subdir_pkls += '_GRP_BY_OUTG_AND_XFMR'
elif grp_by_col=='trsf_pole_nb':
    save_subdir_pkls += '_GRP_BY_XFMR'
elif grp_by_col=='outg_rec_nb':
    save_subdir_pkls += '_GRP_BY_OUTG'
elif grp_by_col==['trsf_pole_nb', 'no_outg_rec_nb']:
    save_subdir_pkls += '_GRP_BY_NO_OUTG_AND_XFMR'
else:
    assert(0)
#-----
save_dir_base_pkls = os.path.join(data_base_dir, save_subdir_pkls)
save_dir_pkls = os.path.join(save_dir_base_pkls, f'outg_td_window_{days_min_outg_td_window}_to_{days_max_outg_td_window}_days')
#-----
if save_dfs_to_pkl and not os.path.exists(save_dir_pkls):
    os.makedirs(save_dir_pkls)

In [ ]:
print(f'save_dir_base_pkls = {save_dir_base_pkls}')
print(f'save_dir_pkls      = {save_dir_pkls}')

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# TODO NEED TO BE AUTOMATED
outg_rec_nb_idx_lvl=-1
trsf_pole_nbs_idx_lvl=0

trsf_pole_nbs_loc = ('index', 'trsf_pole_nb')
# rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')]
# rcpo_df_to_time_infos_on = ['index']
# time_infos_to_rcpo_df_on = ['index']

rcpo_df_to_time_infos_on = [('index', 'trsf_pole_nb'), ('index', 'no_outg_rec_nb')]
time_infos_to_rcpo_df_on = [('index', 'trsf_pole_nb'), ('index', 'no_outg_rec_nb')]

# rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')]
rcpo_df_to_PNs_on = ['index']
PNs_to_rcpo_df_on = ['index']
how='left'

# TODO!~
Should all : be replaced with \: (except. e.g., ?:)
Also, do all . need to be replaced with \. when I mean period instead of any character?!

In [ ]:
patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
   (
       (
           r'(Under Voltage)\s*'\
           r'([0-9a-zA-Z]*)?\s*'\
           r'(\([0-9a-zA-Z\s]*\))\s*'\
           r'([0-9a-zA-Z]*)?\s?'\
           r'(for meter\:?\s*)'\
           r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
           r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
           r'(Voltage out of tolerance)?'
       ), 
       'AMIEndEvents.under_voltage_match_func'
   ), 
   (
       (
           r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
           r'.*'\
           r'(Fail Reason: .*)$'
       ), 
       'AMIEndEvents.last_gasp_reduce_func'
   ), 
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    ('meterN/A', 'meter'),
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    r'N/A', 
    (r'\s{2,}', ' ')
]

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
if not read_dfs_from_pkl:
    start = time.time()
    rcpo_df_OG, ede_typeid_to_reason_df_OG = AMIEDE_DEV.build_reason_counts_per_outage_from_csvs(    
        files_dir=files_dir, 
        file_path_glob=file_path_glob, 
        file_path_regex=file_path_regex, 
        patterns_to_replace=patterns_to_replace, 
        min_outg_td_window=datetime.timedelta(days=days_min_outg_td_window),
        max_outg_td_window=datetime.timedelta(days=days_max_outg_td_window),
        build_ede_typeid_to_reason_df=True, 
        batch_size=batch_size, 
        cols_and_types_to_convert_dict=None, 
        to_numeric_errors='coerce', 
        assert_all_cols_equal=assert_all_cols_equal, 
        include_normalize_by_nSNs=include_normalize_by_nSNs, 
        inclue_zero_counts=inclue_zero_counts, 
        return_multiindex_outg_reason=return_multiindex_outg_reason, 
        return_normalized_separately=return_normalized_separately, 
        verbose=True, 
        n_update=1, 
        grp_by_cols=grp_by_col, 
        is_no_outage=is_no_outage, 
        include_prem_nbs=True, 
        set_faulty_mp_vals_to_nan=False,
        correct_faulty_mp_vals=False
    )
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        rcpo_df_OG.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
        ede_typeid_to_reason_df_OG.to_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))        
else:
    rcpo_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
    ede_typeid_to_reason_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))    

In [ ]:
# #TODO DELETE ME!!!!!!!!!
# rcpo_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_OG.pkl'))
# ede_typeid_to_reason_df_OG = pd.read_pickle(os.path.join(save_dir_pkls, f'ede_typeid_to_reason{naming_tag}_df_OG.pkl'))

In [ ]:
reason_to_ede_typeid_df = AMIEndEvents.invert_ede_typeid_to_reason_df(ede_typeid_to_reason_df_OG)
rcpo_df = rcpo_df_OG.copy()

In [ ]:
# READ IN mp_df_curr_hist, which was saved for 0-30 days
save_dir_pkls_00_30 = os.path.join(save_dir_base_pkls, f'outg_td_window_0_to_30_days')
assert(os.path.isdir(save_dir_pkls_00_30))
assert(os.path.exists(os.path.join(save_dir_pkls_00_30, f'mp{naming_tag}_df_hist.pkl')))
assert(os.path.exists(os.path.join(save_dir_pkls_00_30, f'mp{naming_tag}_df_curr.pkl')))
#-------------------------
mp_df_curr_hist = {}
mp_df_curr_hist['mp_df_hist'] = pd.read_pickle(os.path.join(save_dir_pkls_00_30, f'mp{naming_tag}_df_hist.pkl'))
mp_df_curr_hist['mp_df_curr'] = pd.read_pickle(os.path.join(save_dir_pkls_00_30, f'mp{naming_tag}_df_curr.pkl'))

In [ ]:
# FUCK
# CHANGE rcpo_df_to_time_infos_on!

In [ ]:
paths = Utilities.find_all_paths(base_dir=files_dir, glob_pattern=file_path_glob)
time_infos_df = AMIEDE_DEV.get_no_outg_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEDE_DEV.find_summary_file_from_csv(x) for x in paths], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='t_max', 
    make_addtnl_groupby_idx=True, 
    include_summary_paths=False
)
if save_dfs_to_pkl:
    time_infos_df.to_pickle(os.path.join(save_dir_pkls, f'time_infos{naming_tag}_df.pkl'))

In [ ]:
#TODO!!!!!
# if grp_by_col=='trsf_pole_nb':
if not read_dfs_from_pkl:
    start=time.time()
    #-------------------------
    rcpo_df_raw  = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df, 'counts', droplevel=True)
    #-----
    rcpo_df_raw = add_xfmr_active_SNs_to_rcpo_df(
        rcpo_df=rcpo_df_raw, 
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        set_xfmr_nSNs=True, 
        include_active_xfmr_PNs=True, 
        df_mp_curr=mp_df_curr_hist['mp_df_curr'],
        df_mp_hist=mp_df_curr_hist['mp_df_hist'], 
        time_infos_df=time_infos_df, 
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=dict(assert_all_trsf_pole_nbs_found=False), 
        xfmr_SNs_col='_xfmr_SNs', 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_PNs_col='_xfmr_PNs', 
        xfmr_nPNs_col='_xfmr_nPNs', 
    )
    #-------------------------
    rcpo_df_norm = MECPODf.project_level_0_columns_from_rcpo_wide(rcpo_df, 'counts_norm', droplevel=True)
    #-----
    assert('_xfmr_nSNs' in rcpo_df_raw.columns)
    assert('_xfmr_SNs' in rcpo_df_raw.columns)
    rcpo_df_norm_by_xfmr_nSNs = build_rcpo_df_norm_by_xfmr_active_nSNs(
        rcpo_df_raw=rcpo_df_raw, 
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_SNs_col='_xfmr_SNs', 
        other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
        drop_xfmr_nSNs_eq_0=True, 
        new_level_0_val='counts_norm_by_xfmr_nSNs', 
        remove_SNs_cols=False, 
        df_mp_curr=mp_df_curr_hist['mp_df_curr'],
        df_mp_hist=mp_df_curr_hist['mp_df_hist'], 
        time_infos_df=time_infos_df,
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=dict(assert_all_trsf_pole_nbs_found=False)
    )
    #-------------------------
    icpo_df_raw = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_raw, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=False, 
        SNs_tags=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs']
    )

    icpo_df_norm = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_norm, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_nSNs', 
        SNs_tags=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs']
    )

    icpo_df_norm_by_xfmr_nSNs = MECPODf.convert_rcpo_to_icpo_df(
        rcpo_df=rcpo_df_norm_by_xfmr_nSNs, 
        reason_to_ede_typeid_df=reason_to_ede_typeid_df, 
        is_norm=True, 
        counts_col='_xfmr_nSNs', 
        SNs_tags=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs']
    )
    #-------------------------
    print(time.time()-start)
    #-------------------------
    if save_dfs_to_pkl:
        #-------------------------
        rcpo_df_raw.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_raw.pkl'))
        rcpo_df_norm.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm.pkl'))
        rcpo_df_norm_by_xfmr_nSNs.to_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
        #-----
        icpo_df_raw.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_raw.pkl'))
        icpo_df_norm.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm.pkl'))
        icpo_df_norm_by_xfmr_nSNs.to_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
else:
    rcpo_df_raw = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_raw.pkl'))
    rcpo_df_norm = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm.pkl'))
    rcpo_df_norm_by_xfmr_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'rcpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))
    #-----
    icpo_df_raw = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_raw.pkl'))
    icpo_df_norm = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm.pkl'))
    icpo_df_norm_by_xfmr_nSNs = pd.read_pickle(os.path.join(save_dir_pkls, f'icpo{naming_tag}_df_norm_by_xfmr_nSNs.pkl'))

In [ ]:
assert(0)

In [ ]:
rcpo_dfs_name_outg          = 'rcpo_df_norm_by_xfmr_nSNs'
rcpo_dfs_name_no_outg       = 'rcpo_df_norm_by_xfmr_nSNs'
rcpo_dfs_name_no_outg_prstn = 'rcpo_df_norm_by_xfmr_nSNs'

mecpo_idx_for_ordering = 0

#-------------------------
icpo_dfs_name_outg          = 'i'+rcpo_dfs_name_outg[1:]
icpo_dfs_name_no_outg       = 'i'+rcpo_dfs_name_no_outg[1:]
icpo_dfs_name_no_outg_prstn = 'i'+rcpo_dfs_name_no_outg_prstn[1:]

In [ ]:
mecpo_coll_full
mecpo_coll_no_outg
mecpo_coll_no_outg_prstn

In [ ]:
include_power_down_minus_up=True
#-------------------------
#-------------------------
# Remove all reasons containing 'cleared'
mecpo_coll_full.remove_reasons_from_all_rcpo_dfs(['.*cleared.*', '.*Test Mode.*'])
mecpo_coll_no_outg.remove_reasons_from_all_rcpo_dfs(['.*cleared.*', '.*Test Mode.*'])

#-------------------------
# Combine reasons using the standard combine (see dflt_patterns_and_replace in MECPODf.combine_cpo_df_reasons
#   for the list of default patterns_and_replace)
mecpo_coll_full.combine_reasons_in_all_rcpo_dfs(initial_strip=True, initial_punctuation_removal=True)
mecpo_coll_no_outg.combine_reasons_in_all_rcpo_dfs(initial_strip=True, initial_punctuation_removal=True) 

#-------------------------
# Build power down minus power up counts
if include_power_down_minus_up:
    mecpo_coll_full.delta_cpo_df_reasons_in_all_rcpo_dfs(
        reasons_1='Primary Power Down',
        reasons_2='Primary Power Up',
        delta_reason_name='Power Down Minus Up'
    )
    mecpo_coll_no_outg.delta_cpo_df_reasons_in_all_rcpo_dfs(
        reasons_1='Primary Power Down',
        reasons_2='Primary Power Up',
        delta_reason_name='Power Down Minus Up'
    )